In [1]:
! lamin load scprint

💡 found cached instance metadata: /home/ml4ig1/.lamin/instance--jkobject--scprint.env
💡 loaded instance: jkobject/scprint
💡 loaded instance: jkobject/scprint


In [17]:
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import ModelCheckpoint, StochasticWeightAveraging, EarlyStopping, LearningRateMonitor, LearningRateFinder

seed_everything(42, workers=True)

from scprint import scPrint
from scprint.trainer import TrainingMode
from scdataloader import DataModule
import pandas as pd

import torch
torch.set_float32_matmul_precision('medium')

%load_ext autoreload
%autoreload 2

Global seed set to 42


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

  %reload_ext autoreload


In [3]:
# TODO: drop tissue & dev stage until part or is taken in account

hierarchical_labels = [
    "cell_type_ontology_term_id", #1
    # "tissue_ontology_term_id",
    "disease_ontology_term_id", # 2
#    "development_stage_ontology_term_id",
    "assay_ontology_term_id", #3
    'self_reported_ethnicity_ontology_term_id', #4
]
labels_to_pred = hierarchical_labels+[
    'sex_ontology_term_id', #5
    "organism_ontology_term_id", #6
]
all_labels = labels_to_pred+[
    #'dataset_id',
    'cell_culture',
    "heat_diff",
    "total_counts",
    "nnz",
    "dpt_group",
]

gene_emb = '../data/temp/embeddings.parquet'
d_model=128

In [4]:
# we might want not to order the genes by expression (or do it?)
# we might want to not introduce zeros and 

In [12]:
datamodule = DataModule(
    collection_name="preprocessed dataset",
    gene_embeddings=gene_emb,
    all_labels=all_labels,
    hierarchical_labels=hierarchical_labels,
    organisms=["NCBITaxon:9606"],
    how="most expr",
    max_len=1000,
    add_zero_genes=100,
    label_to_weight=labels_to_pred,
    label_to_pred=labels_to_pred,
    batch_size=64,
    num_workers=16,
    train_oversampling=1.7,
    validation_split=0.2,
    do_gene_pos='../data/main/biomart.parquet',
    test_split=0.1)
testfiles = datamodule.setup()

won't do any check but we recommend to have your dataset coming from local storage

0.0% are aligned
total dataset size is 104.788359405 Gb
---
dataset contains:
     5527449 cells
     70116 genes
     11 labels
     1 organisms
dataset contains 230 classes to predict

seeing a string: loading gene positions as biomart parquet file
these files will be considered test datasets:
    /home/ml4ig1/scprint/.lamindb/BljRloq1xjcxRNDpejzI.h5ad
perc test:  0.002277723412735242


In [18]:
embeddings = pd.read_parquet(gene_emb).loc[model.genes]
sembeddings = torch.nn.AdaptiveAvgPool1d(d_model)(
    torch.tensor(embeddings.values)
)

In [23]:
sembeddings[0]

tensor([ 3.7021e-03,  1.9993e-02, -3.4491e-03,  1.1713e-02,  7.6410e-03,
         3.8214e-02, -2.5759e-03,  1.4394e-02,  1.6626e-02, -2.5141e-02,
        -3.0845e-02, -1.7926e-02,  2.9747e-03,  1.4398e-02, -3.4161e-02,
         1.1524e-02,  8.0428e-03,  1.2203e-02, -4.4529e-03,  1.6340e-02,
        -2.9258e-02,  2.1006e-02, -4.2241e-02, -4.2910e-02,  4.3020e-02,
        -3.0580e-02, -3.9011e-02,  2.0069e-02,  1.7439e-02, -1.7942e-02,
         1.1015e-02, -1.2625e-02, -8.5026e-03, -7.2202e-03,  4.9086e-03,
        -1.3565e-03,  2.5166e-02,  3.2748e-02, -1.4290e-03,  8.9639e-03,
         1.7316e-02,  2.2168e-02,  1.0759e-02, -2.9722e-02,  1.0154e-02,
         8.2518e-03,  3.5274e-02, -5.7122e-03, -2.0104e-02, -2.9521e-02,
         1.0372e-02,  5.1883e-03, -1.3417e-02,  1.7161e-02,  5.3995e-03,
        -1.1004e-02,  1.7635e-02,  5.9054e-02, -1.6850e-02,  1.8962e-02,
        -8.4115e-03,  1.8526e-02,  2.9677e-02, -1.3948e-02,  8.8496e-03,
        -1.7609e-02,  8.1042e-03, -1.1275e-03,  3.8

In [32]:
model = scPrint(
    genes = datamodule.genes,
    d_model = d_model,
    nhead = 4,
    d_hid = d_model,
    nlayers = 4,
    layers_cls = [d_model],
    labels = datamodule.labels,
    cls_hierarchy = datamodule.cls_hierarchy,
    dropout= 0.1,
    transformer = "flash",
    precpt_gene_emb = gene_emb,
    gene_pos_enc = datamodule.gene_pos,
    mvc_decoder = "inner product",
    label_decoders = datamodule.decoders,
    fused_dropout_add_ln = False
)

here
adding
tensor([[ 0.0037,  0.0200, -0.0034,  ...,  0.0202,  0.0345, -0.0313],
        [-0.0007,  0.0393, -0.0069,  ...,  0.0281,  0.0400, -0.0190],
        [-0.0175,  0.0621, -0.0132,  ...,  0.0161,  0.0288, -0.0111],
        ...,
        [-0.0217,  0.0621, -0.0146,  ...,  0.0505,  0.0368, -0.0321],
        [ 0.0173,  0.0463, -0.0083,  ...,  0.0037,  0.0115, -0.0153],
        [ 0.0525,  0.0525, -0.0402,  ...,  0.0067,  0.0007, -0.0331]])
scPrint(
  (gene_encoder): GeneEncoder(
    (embedding): Embedding(33890, 128)
    (enc_norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (expr_encoder): ContinuousValueEncoder(
    (linear1): Linear(in_features=1, out_features=128, bias=True)
    (activation): ReLU()
    (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (label_encoder): Cat

In [7]:
# create a function to transform an scGPT checkpoint to an scPrint's
# ckpt = torch.load("../../scGPT/save/model_e6.pt")
# scPrint.load_from_checkpoint("../../scGPT/save/model_e6.pt")

In [5]:
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.loggers import WandbLogger

wandb_logger = WandbLogger(project="scprint_test", save_dir="../data/tensorboard")
wandb_logger.watch(model, log='all', log_freq=50, log_graph=True)

#tlogger = TensorBoardLogger(save_dir="../data/tensorboard")
#tlogger.log_graph(model, i)


2024-02-19 16:41:43,615:ERROR - Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jkobject (ml4ig). Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [9]:
#from lightning.pytorch.profilers import PyTorchProfiler
#pytorch_prof = PyTorchProfiler("../data/tensorboard", emit_nvtx=False, group_by_input_shape=True, record_shapes=True, profile_memory=True, with_stack=True, on_trace_ready=torch.profiler.tensorboard_trace_handler("../data/tensorboard/"),)

In [6]:
chckp = ModelCheckpoint(monitor="val_loss", save_top_k=-1)
trainingmode = TrainingMode(do_denoise=True, noise=[0.3], do_cce=True, cce_sim=0.5, do_ecs=True, ecs_threshold = 0.3, ecs_scale = 1.0, do_mvc=False, do_adv_cls=False, do_next_tp=False, class_scale = 5000.0, mask_ratio=[0.15, 0.3])
es = EarlyStopping(patience=10, monitor='val_loss')
swa = StochasticWeightAveraging(swa_lrs= 0.01)
lrm = LearningRateMonitor(logging_interval="step")
lrf = LearningRateFinder(mode="exponential",)
# TODO: to check that the class hierarchy are really ordered from 1-2-3-4... as well (oredered dict)
trainer = Trainer(precision="16-mixed", gradient_clip_val=10, max_time={"hours": 3}, limit_train_batches=5000, limit_test_batches=0.03, limit_val_batches=1000, callbacks=[chckp, trainingmode, es, lrm], logger=wandb_logger, accumulate_grad_batches=2, reload_dataloaders_every_n_epochs=1) #detect_anomaly=True, fast_dev_run=20, overfit_batches=10, limit_train_batches=1, limit_val_batches=0


Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [21]:
model.lr, model.hparams.lr

(0.001, 0.001)

In [9]:
# sanity. should be overfiting.
trainer = Trainer(precision="16-mixed", limit_train_batches=500, limit_val_batches=0, check_val_every_n_epoch=1000, log_every_n_steps=1000) #logger=wandb_logger)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [10]:
trainer.fit(model, datamodule=datamodule)

these files will be considered test datasets:
    /home/ml4ig1/scprint/.lamindb/BljRloq1xjcxRNDpejzI.h5ad
perc test:  0.00226129182087695


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                    | Params
----------------------------------------------------------
0 | gene_encoder  | GeneEncoder             | 4.3 M 
1 | expr_encoder  | ContinuousValueEncoder  | 512   
2 | pos_encoder   | PositionalEncoding      | 0     
3 | label_encoder | CategoryValueEncoder    | 1.3 K 
4 | time_encoder  | ContinuousValueEncoder  | 512   
5 | transformer   | FlashTransformerEncoder | 793 K 
6 | expr_decoder  | ExprDecoder             | 50.6 K
7 | cls_decoders  | ModuleDict              | 130 K 
8 | mvc_decoder   | MVCDecoder              | 99.3 K
----------------------------------------------------------
1.1 M     Trainable params
4.3 M     Non-trainable params
5.4 M     Total params
21.656    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir="../data/tensorboard"

In [21]:
#wandb_logger.finalize(status="aborted")
torch.cuda.empty_cache()

In [ ]:
# TODO: test unseen genes (do we see much being kept after filtering and stuff) (0.5 day)
----
# TODO: connect with maestro people to ask for longer compute time 
# TODO: do the same to jean zay (0.5 day)
------
# TODO: make a model benchmark package (continue from where I left off) (4 days)
# TODO: make a task function & make a benchmark function (1 day) (*denoising, *classification, *embeddings, *perturbation prediction)
------
# TODO: debug the gene embedding creation
# TODO: create embedding & make it work for the 4-5 species in the dataset (1 days) 
# TODO: debug the timepoint problem (2 days)
# TODO: find the neighboors and next time point cells (1 days)
# TODO: create a version with next time point and neighboors task (1 days)
# TODO: make a trajectory prediction task (predict future cell type/s, expression) and benchmark (similarity to known future cell, similarity to known future expression) (1 days)
------
# TODO: run a large training on maestro (0.5 day)
------
# TODO: add KO & drug datasets
# TODO: create a version with KO and drug effect prediction